In [ ]:
# Dependencies
%pip install mplfinance
import requests
import json
from pymongo import MongoClient
import mplfinance as mpf

In [65]:
# Import the API key
from kirsten_api_keys import alpha_vantage_api

In [66]:
# Top gainers/losers/volume api parameters
first_function = 'TOP_GAINERS_LOSERS'


In [67]:
# Make the api call to retrieve the top 20 gainers, losers, and most volume traded
first_url = f'https://www.alphavantage.co/query?function={first_function}&apikey={alpha_advantage_api}'
first_r = requests.get(first_url)
first_data = first_r.json()

In [ ]:
# view the information and select 8 stocks to analyze
first_data

Group 4 will be analyzing NVDA, WOK, YIBO, AVGX, DWTX, ELAB, AKRO, NRO

In [69]:
# Target api parameters
function = 'TIME_SERIES_DAILY'
symbol = ['NVDA','WOK','YIBO','AVGX','DWTX','ELAB','AKRO','NRO']
output = 'full'



In [ ]:
# Iterate through the list of stock tickers and make retrieve the data for each stock. Stored in api_stock_data
api_stock_data = []
for stock in symbol:
    url = f'https://www.alphavantage.co/query?function={function}&symbol={stock}&outputsize={output}&apikey={alpha_advantage_api}'
    r = requests.get(url)
    data = r.json()
    for date,info in data['Time Series (Daily)'].items():
        info['date'] = date
        info['symbol'] = stock
        api_stock_data.append(info)

In [ ]:
# View the information from the retrieved data
api_stock_data

In [10]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [11]:
# assign the project3_group4 database to a variable name
db = mongo.project3_group4

In [ ]:
# Insert the stock data into the stock_data collection within the database
db.stock_data.insert_many(api_stock_data)

In [ ]:
# Create a list out of the stock data that was imported into the mongoDB data base and collection
# Assign a variable to the list
mongo_stock_data = list(db.stock_data.find())
# View the information
mongo_stock_data

Mongo_stock_data will be the collection that holds all stock information for all 8 stocks that we will be analysis. 

In [ ]:
# Save the json file to the Data folder (which will be used for the project analysis)
#  NOTE: THE DATA ANALYSIS WILL BE PERFORMED ON THE STOCK DATA BETWEEN 1/28/2025 AND THE LAST 20 YEARS OR SO ON A DAILY INTERVAL.
# STOCK DATA IS ONLY AVAILABLE FOR BUSINESS DAYS (NOT WEEKENDS OR FEDERAL HOLIDAYS) AS THE STOCK MARKET IS CLOSED ON THOSE DAYS.

#COMMENT OUT THIS PART WITH GROUP

#import json

#file_path = '../Data/mongo_stock_data.json'

#with open(file_path, 'w') as json_file:
    #json.dump(mongo_stock_data, json_file, default= str)

In [76]:
import pandas as pd
import json

# Load the JSON file
file_path = '../Data/mongo_stock_data.json'

with open(file_path, 'r') as json_file:
    stock_data = json.load(json_file)

# Convert to DataFrame
df = pd.DataFrame(stock_data)

# Display the first few rows
df.head()

,1. open,2. high,3. low,4. close,5. volume,date,symbol
0,121.8100,129.0000,116.2500,128.9900,574006633,2025-01-28,NVDA
1,124.8000,128.4000,116.7000,118.4200,818830895,2025-01-27,NVDA
2,148.3700,148.9700,141.8800,142.6200,234657635,2025-01-24,NVDA
3,145.0500,147.2300,143.7200,147.2200,155915549,2025-01-23,NVDA
4,144.6600,147.7900,143.6700,147.0700,237651395,2025-01-22,NVDA


In [93]:
# Filter for specific stocks
df_filtered = df[df['symbol'].isin(['YIBO', 'AVGX'])]

# Display filtered data
df_filtered.head()

,1. open,2. high,3. low,4. close,5. volume,date,symbol
6456,5.5800,6.1200,4.2600,5.1400,55408228,2025-01-28,YIBO
6457,2.7000,17.4700,2.7000,3.0800,35240720,2025-01-27,YIBO
6458,3.2000,3.2000,2.0800,2.1500,53216,2025-01-24,YIBO
6459,2.9900,3.3100,2.9900,3.0800,12620,2025-01-23,YIBO
6460,3.0800,3.0800,3.0600,3.0600,803,2025-01-22,YIBO


In [94]:
df_filtered['date'] = pd.to_datetime(df_filtered['date'])

# Set index as date
df_filtered.set_index('date', inplace=True)

# Sort by date
df_filtered = df_filtered.sort_index()

df_filtered.head()

C:\Users\kcalo\AppData\Local\Temp\ipykernel_10944\160530148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date'])


,1. open,2. high,3. low,4. close,5. volume,symbol
date,,,,,,
2024-01-25,4.0000,4.0000,2.4000,2.7900,875455,YIBO
2024-01-26,2.9600,2.9600,1.8600,2.7200,386336,YIBO
2024-01-29,2.4700,2.7600,2.3000,2.6000,79493,YIBO
2024-01-30,2.2500,2.6999,2.2500,2.6600,47410,YIBO
2024-01-31,2.3800,2.7200,2.3800,2.4200,61474,YIBO


In [102]:
import pandas as pd

# Define 20-year cutoff date
start_date = pd.to_datetime("2005-01-28")

# Filter dataset
df_filtered = df_filtered[df_filtered.index >= start_date]

# Check data range
print(df_filtered.index.min(), df_filtered.index.max())
(df_filtered)

2024-01-25 00:00:00 2025-01-28 00:00:00


,1. open,2. high,3. low,4. close,5. volume,symbol
date,,,,,,
2024-01-25,4.0000,4.0000,2.4000,2.7900,875455,YIBO
2024-01-26,2.9600,2.9600,1.8600,2.7200,386336,YIBO
2024-01-29,2.4700,2.7600,2.3000,2.6000,79493,YIBO
2024-01-30,2.2500,2.6999,2.2500,2.6600,47410,YIBO
2024-01-31,2.3800,2.7200,2.3800,2.4200,61474,YIBO
...,...,...,...,...,...,...
2025-01-24,35.4900,36.9599,34.7500,35.4600,645810,AVGX
2025-01-27,26.1300,27.8900,21.5000,23.2600,4540142,AVGX
2025-01-27,2.7000,17.4700,2.7000,3.0800,35240720,YIBO


In [107]:
df_copy = df_filtered.copy().reset_index()  # Bring 'date' back as a column

In [108]:
df_copy.rename(columns={
    'date': 'Date',  # Now 'date' exists as a column, rename it
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. volume': 'Volume'
}, inplace=True)

In [109]:
print(df_copy.columns)  # Ensure 'Date' appears in the list
print(df_copy.head())  # Check if 'Date' contains values


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'symbol'], dtype='object')
        Date    Open    High     Low   Close  Volume symbol
0 2024-01-25  4.0000  4.0000  2.4000  2.7900  875455   YIBO
1 2024-01-26  2.9600  2.9600  1.8600  2.7200  386336   YIBO
2 2024-01-29  2.4700  2.7600  2.3000  2.6000   79493   YIBO
3 2024-01-30  2.2500  2.6999  2.2500  2.6600   47410   YIBO
4 2024-01-31  2.3800  2.7200  2.3800  2.4200   61474   YIBO


In [105]:
df_copy.columns = [col.split('. ')[-1].capitalize() for col in df_copy.columns]


In [110]:
print(df_copy.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'symbol'], dtype='object')


In [111]:
df_copy['Date'] = pd.to_datetime(df_copy['Date']).dt.strftime('%Y-%m-%d')

In [112]:
df_copy.to_json("kirsten_cleaned_stock_data.json", orient="records", indent=4)